# Examen de cómputo matricial equipo SVD

**Fecha**

19 de Abril de 2020

**Objetivo**

Programar en el lenguaje R el método de eliminación por bloques para la solución de un sistema de ecuaciones $$Ax=b$$ mediante el método de descomposición en valores singulares (DVS o SVD por siglas en inglés), a través del Algoritmo One-Sided Jacobi.

**Especificaciones de ambiente común de trabajo**

Para trabajar y a efecto de tener un entorno común de trabajo para el desarrollo del proyecto, por favor emplear la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 1.1 Generación de índices

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices


}

### 1.2 Verificación de ortogonalidad entre vectores

In [2]:
ortogonal <- function(u,v,TOL=10^-8){
  # Verifica si dos vectores son ortogonales, arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
    # u, v como vectores de la misma dimensión.Y un valor real de tolerancia TOL(10^-8).
    # Nota: Se sugiere una TOL mayor a 10^-32.
  # Returns: 
    # Valor booleano 0 (no son ortongoales), 1 (son ortogonales)
    if ( norm(u,type ="2") < TOL | norm(v,type ="2") < TOL){ret<-0} 
    else{ 
        if( (u%*%v)/(norm(u,type ="2")*norm(v,type ="2")) < TOL){ret<-1}
        else{ret<-0}  
    }
  ret
}

### 1.3 Función signo

In [3]:
signo<-function(x) {
  # Indica el signo de un número x
  # Args: 
  #    x (numeric): número a revisar
  # Returns:
  #    1 si el número es positivo o cero
  #    -1 si el número es negativo
  
  ifelse(x<0,-1,1)
  }

### 1.4 Solver dada descomposición SVD

In [4]:
solver <- function(U,S,V,b){
    # Construye la solución de un sistema de ecuaciones a partir de matrices 
    # U, S, V, y vector b. Se asume que S es diagonal. 
    # Para ello resuelve S d = U^Tb, para construir x=Vd.
    # Notas:
    # 1) Se utilizó la función backsolve para resolver el sistema triangular.
    # 2) Al ser S diagonal, es indistinto si usar un solver para matrices traingulares inferiores o superiores.
    # Args: 
    #  	    U (mxm),V(nxn), S(mxn) matriz diagonal y b (m) un vector.
    # Returns: 
    #      x vector (m)
  d = backsolve(S, t(U)%*%b)
  x = V%*%d
  return(x)
}

## 2. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 One-sided Jacobi numerical aproximación

In [5]:
svd_jacobi_aprox <- function(A,TOL,maxsweep){
    # Función que calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representan a las matriz S(mxm) matriz diagonal,el segundo a la matriz U(nxm)
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    
    #dimensiones
    n<-dim(A)[2] #numero de columnas
    m<-dim(A)[1] #numero de filas
    nmax<-n*(n-1)/2

    #inicialza valores del ciclo
    ak<-A
    vk<-diag(n)
    sig <- NULL
    uk <- ak
    num_col_ortogonal<-0
    k<-0

    while(k<=maxsweep & num_col_ortogonal<nmax){
    num_col_ortogonal<-0
    ind <- indices(n)
    for(i in 1:nmax){
      col_j<-ak[,ind[[i]][[1]][2]]
      col_i<-ak[,ind[[i]][[1]][1]]
    
      #comprueba ortogonalidad  
      if(ortogonal(col_i,col_j,TOL)==1){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-col_i%*%col_i
        b<-col_j%*%col_j
        c<-col_i%*%col_j
        
        if(c<TOL){break}
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        temp<-ak[,ind[[i]][[1]][1]] 
        ak[,ind[[i]][[1]][1]]<-c(cs)*temp-c(sn)*ak[,ind[[i]][[1]][2]]
        ak[,ind[[i]][[1]][2]]<-c(sn)*temp+c(cs)*ak[,ind[[i]][[1]][2]]
        
        
        #actualiza las columnas de la matriz vk
        temp<-vk[,ind[[i]][[1]][1]] #cambio
        vk[,ind[[i]][[1]][1]]<-c(cs)*temp-c(sn)*vk[,ind[[i]][[1]][2]]
        vk[,ind[[i]][[1]][2]]<-c(sn)*temp+c(cs)*vk[,ind[[i]][[1]][2]]             
       }#cierra else
    }#cierra for
  k<-k+1
 }#cierra while
    
    #Obtener sigma
    sig<-apply(ak, 2, function(x){norm(x,"2")})

    #Obtener U
    for(i in 1:n){
        if (sig[i]<TOL){
            uk[,i]<-0  
        } else{
        uk[,i] <- ak[,i]/sig[i]
        }
    }

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    index <- order(sig,decreasing = TRUE)
    vk <- vk[,index]
    S <- diag(sig[index])
    uk <- uk[,index]

    list(S = S, U = uk, V= vk)
 }   

In [6]:
#Ejemplo
#parametros de entrada
A<-matrix(c(7,0,1,-5.1,0.4,1,3,5,8,-1,6,-1),nrow=4)
TOL<-10**-8
maxsweep<-20
#Función
svd<-svd_jacobi_aprox(A,TOL,maxsweep)

In [7]:
svd

12.54399,0.000000,0.000000
0.00000,5.929587,0.000000
0.00000,0.000000,4.545133
0.84707087,0.06745833,0.06800458
-0.06149602,0.16864581,0.14000551
0.41970508,0.50593744,-0.67031182
-0.32021371,0.84322907,-0.72557280
0.6363437,0,0.7714057
0.0000000,1,0.0000000
0.7714057,0,-0.6363437


### 2.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm

In [8]:
sel_solver<-function(A,b,TOL=10**-8,maxsweep=20){
    #Función resuelve un sistema de ecuaciones lineales (SEL) utilizando la descomposición SVD
    #por medio del método de One-sided Jacobi 
    #El SEL es de la forma Ax=b
    # Args: 
    #    A (float): matriz de incógnitas del SEL
    #    b (float): vector de igualdada del sistema
    #    TOL (numeric): controla la convergencia del método
    #    maxsweep (int): número máximo de sweeps 
    #Returns: x (float): vector solución 

    svd<-svd_jacobi_aprox(A,TOL,maxsweep)
    x<-solver(svd$U,svd$S,svd$V,b)
    x
}

In [9]:
A<-matrix(c(-1,3,2,2,1,3),nrow=3)
b<-c(5,7,12)
x<-sel_solver(A,b,maxsweep=40)

In [10]:
A%*%x

5
7
12


### 2.3 Eliminación por bloques basada en solver linear que usa SVD

In [11]:
bloques<-function(A,b,corte) {
  #Función que genera los bloques de la matriz A y el vector respuesta b
  #Args: A (matriz) - matriz inicial(n*n)
  #      b (vector) - Solución de Ax = b , (nx1)
  #      corte (int) - tamaño del bloque (valor entre 1 y n)
  #Returns: lista con la matriz A dividida en 4 bloques: A11,A12, A21, A22 y el vector b
  #         dividido en 2 bloques b1, b2

  a11 <- A[c(1:corte),c(1:corte)]
  a12 <- A[c(1:corte),c((corte+1):dim(A)[2])]
  a21 <- A[c((corte+1):dim(A)[1]),c(1:corte)]
  a22 <- A[c((corte+1):dim(A)[1]),c((corte+1):dim(A)[2])]

  b1 <- b[1:(corte)]
  b2 <- b[((corte)+1):length(b)]

  list(A11 = a11,A12 =a12, A21=a21, A22=a22, b1 = b1, b2 = b2)
} 

In [12]:
#Prueba
A = matrix(sample(0:50,7*7,replace=TRUE), c(7,7)) 
b = c(1:dim(A)[1])

A
b
bloques(A,b,4)


0,14,42,35,45,7,24
25,21,34,33,6,39,5
43,12,0,26,0,29,23
21,13,26,6,12,28,37
2,10,33,9,46,16,16
41,9,13,19,45,34,17
1,14,43,40,42,17,33


[1] 1 2 3 4 5 6 7

$A11
     [,1] [,2] [,3] [,4]
[1,]    0   14   42   35
[2,]   25   21   34   33
[3,]   43   12    0   26
[4,]   21   13   26    6

$A12
     [,1] [,2] [,3]
[1,]   45    7   24
[2,]    6   39    5
[3,]    0   29   23
[4,]   12   28   37

$A21
     [,1] [,2] [,3] [,4]
[1,]    2   10   33    9
[2,]   41    9   13   19
[3,]    1   14   43   40

$A22
     [,1] [,2] [,3]
[1,]   46   16   16
[2,]   45   34   17
[3,]   42   17   33

$b1
[1] 1 2 3 4

$b2
[1] 5 6 7

Algoritmo

Sean $A$ y $A_{11}$ no singulares.

1) Calcular $A_{11}^{-1}A_{12}$ y $A_{11}^{-1}b_1$ teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales:

Para realizar la multiplicación $A_{11}^{-1}b_1$ definimos $y=A_{11}^{-1}b_1$ y por tanto $A_{11}y = b_1$ ($A_{11}$ es no singular). Así, resolvemos para $y$ el sistema anterior y habremos calculado $A_{11}^{-1}b_1$. Similarmente definimos $Y=A_{11}^{-1}A_{12}$ con lo que se tiene $A_{11}Y=A_{12}$. Resolvemos para $Y \in \mathbb{R} ^{n_1 \times n_1}$ y habremos calculado $A_{11}^{-1}A_{12}$.
2) Calcular el complemento de Schur del bloque $A_{11}$ en $A$: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. Calcular $ \hat{b} = b_2-A_{21}A_{11}^{-1}b_1$.

3) Resolver $Sx_2 = \hat{b}$.

4) Resolver $A_{11}x_1 = b_1-A_{12}x_2$.

In [13]:
install.packages("matrixcalc")
library("matrixcalc")


The downloaded binary packages are in
	/var/folders/4h/pz7sf1h93mn0vsyplpp3jxx40000gn/T//RtmpeHk9Xx/downloaded_packages


In [14]:


eliminacion_bloques <- function(A,b,corte, TOL, maxsweep){
  #Función que realiza el método de eliminación por bloques
  #Args: A (matriz)-  matriz inicial(n*n)
  #      b (vector) - Solución de Ax = b , (nx1)
  #      corte (int) - tamaño del bloque (valor entre 1 y n)
  #      TOL (double) - controla la convergencia del método (valor menor o igual a 10^{-8})
  #      maxsweep (int) - número máximo de sweeps  (valor entre 1 y (n-1)*n/2)
  #Returns: x: vector

  bloq = bloques(A,b,corte)
  if(is.singular.matrix(A,TOL) ==FALSE & is.singular.matrix(bloq$A11,TOL) == FALSE){
    
  y = sel_solver(bloq$A11,bloq$b1,TOL, maxsweep)
      
  Y= bloq$A12
  for(i in 1:dim(bloq$A12)[2]){
    Y[,i] = sel_solver(bloq$A11,bloq$A12[,i],TOL, maxsweep)
  }
      
  S = bloq$A22 - bloq$A21%*%Y 
  b_hat = bloq$b2-bloq$A21%*%y 
  x_2 = sel_solver(S,b_hat,TOL, maxsweep)
  b_hat2 = bloq$b1 -bloq$A12%*%x_2
  x_1 = sel_solver(bloq$A11,b_hat2,TOL, maxsweep)
  x <- c(x_1,x_2)
  x
  }else{
    print("Singulraidad de las matrices involucradas; no hay solucion")
  }
  
}

In [15]:
A = matrix(sample(0:50,7*7,replace=TRUE), c(7,7)) 
b = c(1:dim(A)[1])
TOL = 10**-8
maxsweep=40
corte = 4
eliminacion_bloques(A,b,corte, TOL, maxsweep)

[1]  0.16928928 -0.06695635  0.18048952 -0.14924064 -0.12274792  0.10069851
[7]  0.05013246

## 3 Test

### 3.1 Función error relativo

In [16]:
error_relativo <- function(A,x,b){
    #Error relativo de Ax = b
    #A (matriz) =  matriz de tamaño nxm 
    #x (vector) = vector de tamaño mx1
    #v (vector) = vector de tamaño nx1
    error <- norm((A%*%x-b),type ="2")/norm(b,type ="2")
    error
    
}

In [29]:
devtools::install_github("collectivemedia/tictoc")
library(tictoc)

✔  checking for file ‘/private/var/folders/4h/pz7sf1h93mn0vsyplpp3jxx40000gn/T/RtmpeHk9Xx/remotesc30f4b48d84d/collectivemedia-tictoc-211fe59/DESCRIPTION’
─  preparing ‘tictoc’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘tictoc_1.0.tar.gz’
   


In [34]:
#Prueba con matriz nxn con n = 10
A = matrix(sample(0:50,10*10,replace=TRUE), c(10,10)) 
b = c(sample(0:50,dim(A)[1],replace=TRUE))
corte = 5
TOL = 10^(-8)
maxsweep =30
print("Prueba con una matriz A (10 x 10)")
tic("Tiempo de prueba")
x <- eliminacion_bloques(A,b,corte, TOL, maxsweep)
toc()

[1] "Prueba con una matriz A (10 x 10)"
Tiempo de prueba: 0.197 sec elapsed


In [35]:
error_relativo(A,x,b)

[1] 1.416679

In [36]:
# Cambiando los valores de TOL y maxsweep
TOL = 10^(-22)
maxsweep =10
print("Prueba con una matriz A (10 x 10)")
tic("Tiempo de prueba")
x <- eliminacion_bloques(A,b,corte, TOL, maxsweep)
toc()

[1] "Prueba con una matriz A (10 x 10)"
Tiempo de prueba: 0.122 sec elapsed


In [37]:
error_relativo(A,x,b)

[1] 0.9642487

In [38]:
#Cambiando una vez más los valores de TOL y maxsweep
TOL = 10^(-35)
maxsweep =60
print("Prueba con una matriz A (10 x 10)")
tic("Tiempo de prueba")
x <- eliminacion_bloques(A,b,corte, TOL, maxsweep)
toc()

[1] "Prueba con una matriz A (10 x 10)"
Tiempo de prueba: 0.523 sec elapsed


In [39]:
error_relativo(A,x,b)

[1] 1.434076

In [66]:
#Prueba con matriz nxn con n = 10^2
A = matrix(sample(0:50,10^2*10^2,replace=TRUE), c(10^2,10^2)) 
b = c(sample(0:50,dim(A)[1],replace=TRUE))
corte = 30
TOL = 10^(-8)
maxsweep =50
print("Prueba con una matriz A (10^2 x 10^2)")
tic("Tiempo de prueba")
x <- eliminacion_bloques(A,b,corte, TOL, maxsweep)
toc()

[1] "Prueba con una matriz A (10^2 x 10^2)"
Tiempo de prueba: 156.912 sec elapsed


In [67]:
error_relativo(A,x,b)

[1] 0.8156086

In [68]:
#Cambiando punto de corte de bloques, tolerancia, y maxsweep
corte = 50
TOL = 10^(-15)
maxsweep =20
print("Prueba con una matriz A (10^2 x 10^2)")
tic("Tiempo de prueba")
x <- eliminacion_bloques(A,b,corte, TOL, maxsweep)
toc()

[1] "Prueba con una matriz A (10^2 x 10^2)"
Tiempo de prueba: 136.968 sec elapsed


In [69]:
error_relativo(A,x,b)

[1] 0.5395906

Obs. El nivel de tolerancia no determina al error relativo, sin embargo con el valores pequeños  de maxsweep el error relativo muestra cifras más pequeñas